In [11]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, GPT2Tokenizer, GPT2LMHeadModel
import requests

# CLIP 모델 및 프로세서 로드
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# GPT-2 모델 및 토크나이저 로드
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 이미지 로드 및 전처리
def load_image(image_path=None, url=None):
    if url:
        # URL에서 이미지 로드
        image = Image.open(requests.get(url, stream=True).raw)
    elif image_path:
        # 로컬 경로에서 이미지 로드
        image = Image.open(image_path)
    else:
        raise ValueError("Image path or URL must be provided.")
    return image

# 이미지에서 특징 추출
def extract_image_features(image_path=None, url=None):
    image = load_image(image_path=image_path, url=url)
    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features

# 이미지 특징과 가장 유사한 텍스트 생성
def find_similar_text(image_features, candidate_texts):
    inputs = clip_processor(text=candidate_texts, return_tensors="pt", padding=True)

    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)

    # 이미지 특징과 텍스트 특징 간의 유사도를 계산
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    similarity = torch.matmul(image_features, text_features.T)

    # 가장 유사한 텍스트를 선택
    best_match_index = similarity.argmax().item()
    return candidate_texts[best_match_index]

# 이미지 특징을 기반으로 캡션 생성
def generate_caption_from_image(image_path=None, url=None):
    # 이미지에서 특징 추출
    image_features = extract_image_features(image_path=image_path, url=url)

    
    
    # CLIP 모델로부터 텍스트 후보군 생성 (캡션 후보)
    candidate_texts = [
        "A photo of a cat",
        "A picture of a dog",
        "A landscape with mountains",
        "A person riding a horse",
        "An aerial view of a city",
        "A close-up shot of food",
    ]

    
    
    
    # 이미지 특징과 가장 유사한 텍스트 찾기
    best_caption = find_similar_text(image_features, candidate_texts)

    # GPT-2 모델을 사용하여 이미지 특징 기반으로 캡션 확장
    input_ids = gpt2_tokenizer.encode(best_caption, return_tensors='pt')
    
    #attention_mask = input_ids.ne(gpt2_tokenizer.pad_token_id).long()


    with torch.no_grad():
        # outputs = gpt2_model.generate(input_ids, max_length=30, num_return_sequences=1,
        #                                num_beams=5, early_stopping=True)
        outputs = gpt2_model.generate(input_ids,
                                      pad_token_id = gpt2_tokenizer.eos_token_id,
                                      max_length=50,
                                      num_return_sequences=1,
                                       num_beams=5,
                                       early_stopping=True)

    generated_caption = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_caption

# 예시 이미지 경로 또는 URL
image_path = "/Users/psjj/Downloads/coco2017/train2017/000000000009.jpg"  # 로컬 이미지 경로
url = "http://images.cocodataset.org/val2017/000000039769.jpg"  # URL 이미지

# 캡션 생성
generated_caption = generate_caption_from_image(image_path=image_path)  # 또는 image_path=image_path
# generated_caption = generated_caption.split('.')[0] + '.' # 다 안될 경우 이 코드 추가
print("Generated Caption:", generated_caption)

Generated Caption: A close-up shot of food on the table.

"I don't know what's going on," he said. "I don't know what's going on. I don't know what's going on. I don't know what


In [6]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, GPT2Tokenizer, GPT2LMHeadModel
import torch.nn as nn

# CLIP 및 GPT-2 모델 로드
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# GPT-2의 패딩 토큰 ID 설정
gpt2_model.config.pad_token_id = gpt2_tokenizer.eos_token_id

# 매핑 네트워크 정의 (CLIP 이미지 특징 -> GPT-2 입력으로 변환)
class MappingNetwork(nn.Module):
    def __init__(self, clip_embedding_dim, gpt_embedding_dim):
        super(MappingNetwork, self).__init__()
        self.mapping = nn.Sequential(
            nn.Linear(clip_embedding_dim, gpt_embedding_dim),
            nn.ReLU(),
            nn.Linear(gpt_embedding_dim, gpt_embedding_dim)
        )
    
    def forward(self, clip_features):
        return self.mapping(clip_features)

# 매핑 네트워크 초기화
clip_embedding_dim = 512  # CLIP 이미지 특징 차원
gpt_embedding_dim = gpt2_model.transformer.wte.weight.size(1)  # GPT-2 임베딩 차원
mapping_network = MappingNetwork(clip_embedding_dim, gpt_embedding_dim)

# 이미지 로드 및 전처리
def load_image(image_path=None, url=None):
    if url:
        image = Image.open(requests.get(url, stream=True).raw)
    elif image_path:
        image = Image.open(image_path)
    else:
        raise ValueError("Image path or URL must be provided.")
    return image

# 이미지 특징 추출
def extract_image_features(image_path=None, url=None):
    image = load_image(image_path=image_path, url=url)
    inputs = clip_processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    
    return image_features

# 이미지 특징을 기반으로 캡션 생성
def generate_caption_from_image(image_path=None, url=None):
    # 이미지 특징 추출
    image_features = extract_image_features(image_path=image_path, url=url)

    # CLIP 이미지 특징을 매핑 네트워크를 통해 GPT-2 임베딩으로 변환
    mapped_features = mapping_network(image_features).unsqueeze(0)

    # GPT-2에 프롬프트 설정
    input_ids = gpt2_tokenizer.encode("This image is about", return_tensors="pt")

    # GPT-2 임베딩과 매핑된 CLIP 특징을 결합
    with torch.no_grad():
        outputs = gpt2_model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)

    generated_caption = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_caption

# 이미지 경로 또는 URL 설정
image_path = "/Users/psjj/Downloads/coco2017/train2017/000000000009.jpg"  # 로컬 이미지 경로
url = "http://images.cocodataset.org/val2017/000000039769.jpg"  # COCO 데이터셋 이미지

# 캡션 생성
generated_caption = generate_caption_from_image(image_path=image_path)  # 또는 url=url
print("Generated Caption:", generated_caption)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Caption: This image is about to go viral on social media.

It's a picture of a man with a gun in his hand.

It's a picture of a man with a gun in his hand.

It's a picture of


In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, GPT2Tokenizer, GPT2LMHeadModel
import requests

# CLIP 모델 및 프로세서 로드
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# GPT-2 모델 및 토크나이저 로드
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# 이미지 로드 및 전처리
def load_image(image_path=None, url=None):
    if url:
        # URL에서 이미지 로드
        image = Image.open(requests.get(url, stream=True).raw)
    elif image_path:
        # 로컬 경로에서 이미지 로드
        image = Image.open(image_path)
    else:
        raise ValueError("Image path or URL must be provided.")
    return image

# 이미지에서 특징 추출
def extract_image_features(image_path=None, url=None):
    image = load_image(image_path=image_path, url=url)
    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
    return image_features

# 이미지 특징과 가장 유사한 텍스트 생성
def find_similar_text(image_features, candidate_texts):
    inputs = clip_processor(text=candidate_texts, return_tensors="pt", padding=True)

    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)

    # 이미지 특징과 텍스트 특징 간의 유사도를 계산
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    similarity = torch.matmul(image_features, text_features.T)

    # 가장 유사한 텍스트를 선택
    best_match_index = similarity.argmax().item()
    return candidate_texts[best_match_index]

# 이미지 특징을 기반으로 캡션 생성
def generate_caption_from_image(image_path=None, url=None):
    # 이미지에서 특징 추출
    image_features = extract_image_features(image_path=image_path, url=url)

    # GPT-2 모델에 프롬프트 설정 (프롬프트를 이미지와 관련된 문구로 생성)
    prompt = "This is an image of"

    # GPT-2에 프롬프트를 입력
    input_ids = gpt2_tokenizer.encode(prompt, return_tensors='pt')

    with torch.no_grad():
        outputs = gpt2_model.generate(input_ids,
                                      pad_token_id=gpt2_tokenizer.eos_token_id,
                                      max_length=50,
                                      num_return_sequences=1,
                                      num_beams=5,
                                      early_stopping=True)

    generated_caption = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_caption

# 예시 이미지 경로 또는 URL
image_path = "/Users/psjj/Downloads/coco2017/train2017/000000000009.jpg"  # 로컬 이미지 경로
url = "http://images.cocodataset.org/val2017/000000039769.jpg"  # URL 이미지

# 캡션 생성
generated_caption = generate_caption_from_image(image_path=image_path)  # 또는 image_path=image_path
# generated_caption = generated_caption.split('.')[0] + '.' # 다 안될 경우 이 코드 추가
print("Generated Caption:", generated_caption)
